# Modèles de population structurée

Dans ce TP on va considérer le cas d'une population structurée en sous-populations (ou *demes*) qui échangent de matériel génétique par *migration* des individus.  
Par example, 5 îles de taille $N$ avec un flux de gènes symmétrique (*n-islands model*) ![n-islands model](./nislands.png)

Dans un premier temps on va s'intéresser aux arbres de coalescence.  
Regardons les arbres de coalescence sous deux modèles différents avec taille constante :

1. Population panmictique
2. Population structurée (exemple, le n-islands model)

In [ ]:
library(ape)
library(phyclust)
library(coala)

activate_ms(priority = 500)
n=100 # taille de l'échantillon
loc=10000 # nombre de locus
L=1000 # taille de chaque locus 

N=10000 # taille de la population
mu=2*10^(-8) # taux de mutation par base et par génération
theta=2*N*mu

In [ ]:
# Modele panmictique
model <- coal_model(sample_size=9,loci_number=loc, loci_length=L,ploidy=1) + 
          sumstat_trees()
res <- simulate(model)
tree=read.tree(text=res$trees[[1]])
plot(tree,show.tip.label=TRUE,no.margin=FALSE,direction="downwards")



In [ ]:
model <- coal_model(sample_size=c(3, 3, 3),loci_number=loc, loci_length=L,ploidy=1) + 
  feat_migration(0.1, symmetric = TRUE) + sumstat_trees()
res <- simulate(model)
tree=read.tree(text=res$trees[[1]])
plot(tree,show.tip.label=TRUE,no.margin=FALSE,direction="downwards")

Répétez plusieurs fois la simulation sous le scénario structuré. Il es possible de mettre de couleurs à la base de l'arbre avec le paramètre *tip.color*

In [ ]:
model <- coal_model(sample_size=c(3, 3, 3),loci_number=loc, loci_length=L,ploidy=1) + 
  feat_migration(0.1, symmetric = TRUE) + sumstat_trees()
res <- simulate(model)
tree=read.tree(text=res$trees[[1]])
plot(tree,show.tip.label=TRUE,no.margin=FALSE,direction="downwards", 
    tip.color=c("red", "red", "red", "blue", "blue", "blue", "black", "black", "black"))

** Qu'est-ce qu'on observe ? **

# Calcul des temps de coalescence moyens

Il est possible de calculer la moyenne des temps de coalescence à l'aide de la fonction suivante :

In [ ]:
computeMeanTree <- function(listOfTrees, nbOfTips, plotTree=FALSE){
  # Computes the mean tree from a list. Trees must be in newick format
  # and must have the same number of tips
  #
  # Args:
  #   listOfTrees: list of character containing the trees
  #   nbOfTips: integer, the number of tips the tree have
  #   plotTree: logical, plot the tree or not
  #
  # Returns:
  #   character, the computed mean tree in newick format
  n <- length(listOfTrees)
  branchingTimes <- matrix(rep(seq(1, nbOfTips - 1, 1), n), nrow = n)
  for(i in seq(1, n, 1)){
    newTree <- read.tree(text = listOfTrees[[i]])
    branchingTimes[i, ] <- sort(branching.times(newTree))
  }
  meanTree <- apply(branchingTimes, MARGIN = 2, mean)
  newickTree <- paste("(1:", meanTree[1], ", 2:", meanTree[1], ")", sep = "")
  
  for (i in seq(2, length(meanTree), 1)){
    newickTree <- paste("(", newickTree, ":", meanTree[i] - meanTree[i-1], 
                        ", ", i + 1, ":", meanTree[i], ")", sep = "")
  }
  newickTree <- paste(newickTree, ";", sep = "")
  if(plotTree){
    tree2plot <- read.tree(text = newickTree)
    branching.times(tree2plot)
    plot(tree2plot)
  }
  return(newickTree)
}


Nous allons calculer les temps de coalescence moyens sous un scénario **panmictique** et un scénario **structuré**

In [ ]:
# Modele panmictique
sampleSize <- 9
model <- coal_model(sample_size=sampleSize,loci_number=loc, loci_length=L,ploidy=1) + 
          sumstat_trees()
result <- simulate(model)

listOfTrees <- list()
for(i in seq(1, length(result$trees))){
  listOfTrees <- c(listOfTrees, result$trees[[i]][1])
}
meanTree <- computeMeanTree(listOfTrees = listOfTrees, nbOfTips = sampleSize, plotTree = TRUE)


In [ ]:
# Modele structuré

model <- coal_model(sample_size=c(3, 3, 3),loci_number=loc, loci_length=L,ploidy=1) + 
  feat_migration(0.1, symmetric = TRUE) + sumstat_trees()
result <- simulate(model)

listOfTrees <- list()
for(i in seq(1, length(result$trees))){
  listOfTrees <- c(listOfTrees, result$trees[[i]][1])
}
meanTree <- computeMeanTree(listOfTrees = listOfTrees, nbOfTips = 9, plotTree = TRUE)

# Déficit en hétérozygotes dû à la structuration en sous-populations

Considérons une population divisé en îles et un locus bi-allelique avec les allèles **a** et **A**. Soit $p$ la fréquénce de l'allèle **A** dans la population global et $p_{i}$ la fréquence de l'allèle **A** dans la sous-population $i$. La fréquence de l'allèle **a** dans la sous-population $i$ est donc $(1-p_{i})$. On supose *random-mating* à l'intérieur de chaque sous-population. La fréquence d'hétérozygotes **Aa** à l'intérieur de la sous-population $i$ est égale à :

$$2p_{i}(1-p_{i})$$

La fréquence de l'alléle **A** dans la population globale peut être calculée par :

$$\bar{p}=\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}$$

Sous l'hypothèse de *random-mating* à l'échelle de la population globale, la fréquence d'hétérozygotes devrait être égale à :

$$H_{T}=2\bar{p}(1-\bar{p})$$

Si on calcule la moyenne des fréquences d'hétérozygotes à l'intérieur de chaque île on obtient :

$$
\begin{aligned}
H_{S} &= \frac{1}{n}\displaystyle\sum_{i=1}^{n}2p_{i}(1-p_{i}) = 2\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i} - 2\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}^{2} = 2\bar{p} - 2\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}^{2}\\
      &= 2\bar{p} - 2\bar{p}^{2} + 2\bar{p}^{2} - 2\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}^{2} \\
      &= 2\bar{p}(1 - \bar{p}) + 2\bar{p}^{2} - 2\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}^{2} = H_{T} + 2\bar{p}^{2} - 2\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}^{2} \\
      &= H_{T} + \Bigg(2\frac{1}{n^{2}}\Big(\displaystyle\sum_{i=1}^{n}p_{i}\Big)^2 - 2\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}^{2} \Bigg) \\
      &= H_{T} - 2 \Bigg(\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}^{2} - \Big(\frac{1}{n}\displaystyle\sum_{i=1}^{n}p_{i}\Big)^2 \Bigg) \\
      &= H_{T} - 2 Var(\bar{p})
\end{aligned}
$$


On a donc 
$$H_{S} = H_{T} - 2Var(\bar{p})$$

ce qui veut dire qu'on aura un déficit en hétérozygotes (moins de diversité) à l'intérieur des îles.

Définition: IBS
------------------

On dira que deux gènes sont **IBS** (*Identic By State*) s'ils ont exactement égaux (c'est à dire, s'ils ont le même allèle). Sous certaines hypothèses (*infinite sites model*) cela veut dire qu'il n y a pas eu de mutation dans les lignées des deux gènes entre le présent et l'ancêtre comun le plus récent. 

Définition: le $F_{ST}$
---------------------------

Le $F_{ST}$ est défini en fonction la probabilité de deux gènes d'être "identique" (**IBS**) :

$$F_{ST} = \frac{f_0 - \bar{f}}{1 - \bar{f}}$$

avec $f_0$ la probabilité que deux gènes dans la même sous-population soient identiques et $\bar{f}$ la probabilité que deux gènes choisis au hassard dans l'ensemble de la population soient identiques (voir Herbots 1994 pour plus de détails).

On note que, dans le cas d'un gène biallélique, la probabilité d'être identique peut être calculée en fonction des fréquences des allèles **A** et **a** :


$$
\begin{aligned}
f_{0} &= 1 - Pr(Aa) = 1 - 2p_{i}(1-p_{i}) = 1 - H_{S} \\
\bar{f} &= 1 - 2p(1 - p) = 1 - H_{T}
\end{aligned}
$$

Ce qui nous conduit à une définition équivalente du $F_{ST}$ :

$$\frac{H_{T} - H_{S}}{H_{T}}.$$

Un autre manière de calculer le $F_{ST}$ très utilisée en pratique est :

$$F_{ST} = \frac{Var(\bar{p})}{\bar{p}(1 - \bar{p})}.$$

Dans une population *panmictique* le $F_{ST}$ devrait être égale à zéro (ou proche de zéro en pratique). Cela peut s'expliquer par le fait que la probabilité d'être **IBS** si on prend un sous-ensemble la population (ou un échantillon représentatif) doit être très proche de la probabilité d'être **IBS** dans l'ensemble de toute la population.

Nous allons simuler des données sous un modèle de population panmictique et nous allons calculer le $F_{ST}$. Nous allons faire de même pour un modèle de population structurée (le n-islands model).

Calcul du $F_{ST}$
-----------------------

Compléter la fonction ci-dessous, qui permet de calculer la probabilité que pour un locus donné, deux haplotypes tirés au hasard dans une population soient identiques.


In [ ]:
pIBS.haps <- function(snpData){
    # snpData : tableau de données donnant les allèles de n individus haploïdes (lignes)
    # pour p positions polymorphes (colonnes)
    # samplesIDs : un vecteur avec les indices des séquences à considérer
    n <- dim(snpData)[1]
    p <- dim(snpData)[2]
    hapData <- apply(snpData, 1, paste, collapse="")
    # pour chaque individu, crée une chaîne de caractère donnant son haplotype
    # ...
    # pour chaque haplotype, calcule le nombre d'individus portant cet haplotype
    # pIBS <- ...
    return(pIBS)
}

Tester cette commande (en affichant éventuellement les résultats de calculs intermédiaires) pour les données d’un locus sans recombinaison simulé avec coala (répétez la simulation jusqu'à avoir au moin deux SNP) :

In [ ]:
model=coal_model(sample_size=4,loci_number=1, loci_length=10000,ploidy=1) +
feat_mutation(rate=2) + sumstat_seg_sites()
simResults = simulate(model)
simResults$seg_sites[[1]]

In [ ]:
pIBS.haps(simResults$seg_sites[[1]]$snps)

À l'aide de cette fonction, compléter la fonction suivante qui permet de calculer, à partir de l'attribut *seg_sites* d'un jeu de données simulé par coala, la probabilité **IBS** sur l'ensembe des locus, pour un sous ensemble d'individus donné :

In [ ]:
pIBS.haps.coala <- function(segsites, samplesIDs){
    # segsites : liste de tableaux, telle que renvoyée par l'attribut seg_sites
    # d'un résultat coala
    # samplesIDs : un vecteur avec les indices des individus à considérer
    nbSequences <- length(samplesIDs)
    nbSites <- length(segsites)
    for(i in 1:nbSites){
        # ....
    }
    return(mean(ibs_locus))
}

Tester cette commande pour la simulation coala suivante :

In [ ]:
model <- coal_model(sample_size=10,loci_number=100, loci_length=10000,ploidy=1) + 
                    feat_mutation(rate=2) + sumstat_seg_sites()
simResults <- simulate(model)
pIBS.haps.coala(simResults$seg_sites,1:5)

In [ ]:
pIBS.haps.coala(simResults$seg_sites,6:10)

Pour finir, compléter la fonction ci-dessous qui calcule le $F_{ST}$ à partir de l’attribut seg_sites d’un jeu de données coala :

In [ ]:
computeFst <- function(segsites, popList){
    # segsites : liste de tableaux, telle que renvoyée par l'attribut seg_sites
    # d'un résultat coala
    # popList : liste de vecteurs donnant les indices des individus correspondant
    # à chaque sous-population
    n <- dim(segsites[[1]])[1]
    # probabilité IBS pour deux individus au hasard
    # ...
    # probabilité IBS pour deux individus au hasard dans la même sous-population
    # ...
    for(i in 1:length(popList)){
        # ...
    }
    # calcul du Fst
    # ...
    return(Fst)
}

Tester cette fonction à l'aide de la simulation précédente, comme si les 5 premiers individus venaient d'une sous-population et les 5 derniers d'une autre.

In [ ]:
computeFst(simResults$seg_sites, list(1:3, 4:6, 7:10))

In [ ]:
# Le Fst dans un modèle panmictique devrait être égale à zéro.

# Modèle panmictique (pas de difference entre $f_0_$ et $\bar{f})
model <- coal_model(sample_size=100,loci_number=loc, loci_length=L,ploidy=1) + 
  feat_mutation(rate=theta*L) + sumstat_seg_sites()

res <- simulate(model)
samplesInPopList <- list(1:20, 21:40, 41:60, 61:80, 81:100)
Fst <- computeFst(res$seg_sites, samplesInPopList)
print(paste("Le Fst vaut :", Fst))

In [ ]:
# Modèle de population structurée. n-islands avec n=5 et M=1
structModel <- coal_model(sample_size=c(20, 20, 20, 20, 20), loci_number=loc, loci_length=L,ploidy=1) + 
  feat_migration(1, symmetric = TRUE) + feat_mutation(rate=theta*L) + sumstat_seg_sites()

structRes <- simulate(structModel)
samplesInPopList <- list(1:20, 21:40, 41:60, 61:80, 81:100)
Fst <- computeFst(structRes$seg_sites, samplesInPopList)
print(paste("Le Fst vaut :", Fst))

# Estimation du flux de gènes dans un modèle en îles symmétrique


Il est possible d'estimer le taux de migration ($M = 4Nm$) à partir de la valeur du $F_{ST}$ en se basant sur la formule suivante :

$$F_{ST}\approx \frac{1}{1 + M\frac{d^{2}}{(d-1)^{2}}}$$

avec $d$ le nombre d'îles. Nous allons estimer la valeur du $M$ pour un modèle de population structurée.

In [ ]:
# Fonction pour estimer le flux de gènes
estimate_Fst_M <- function(simResults, samplesInPopList){
  # Fst <- 
  # M <- 
  return(c(Fst, M))
}

In [ ]:
# Modèle de population structurée. n-islands avec n=2 et M=1
structModel_n2 <- coal_model(sample_size=c(20, 20), loci_number=loc, loci_length=L,ploidy=2) + 
  feat_migration(1, symmetric = TRUE) + feat_mutation(rate=theta*L) + sumstat_seg_sites()

structRes <- simulate(structModel_n2)
samplesInPopList <- list(1:20, 21:40)
estimation <- estimate_Fst_M(structRes, samplesInPopList)
Fst <- estimation[1]
M <- estimation[2]
print(paste("Le Fst vaut :", Fst))
print(paste("Le flux de gènes estimé est :", M))

In [ ]:
# Modèle de population structurée. n-islands avec n=5 et M=1
structModel_n5 <- coal_model(sample_size=c(20, 20, 20, 20, 20), loci_number=loc, loci_length=L,ploidy=2) + 
  feat_migration(1, symmetric = TRUE) + feat_mutation(rate=theta*L) + sumstat_seg_sites()

structRes <- simulate(structModel_n5)
samplesInPopList <- list(1:20, 21:40, 41:60, 61:80, 81:100)
estimation <- estimate_Fst_M(structRes, samplesInPopList)
Fst <- estimation[1]
M <- estimation[2]
print(paste("Le Fst vaut :", Fst))
print(paste("Le flux de gènes estimé est :", M))

In [ ]:
# Modèle de population structurée. n-islands avec n=10 et M=1
structModel_n10 <- coal_model(sample_size=c(20, 20, 20, 20, 20, 20, 20, 20, 20, 20), loci_number=loc, 
                              loci_length=L,ploidy=2) + feat_migration(1, symmetric = TRUE) + 
                              feat_mutation(rate=theta*L) + sumstat_seg_sites()
structRes <- simulate(structModel_n10)
samplesInPopList <- list(1:20, 21:40, 41:60, 61:80, 81:100, 101:120, 121:140, 141:160, 161:180, 181:200)
estimation <- estimate_Fst_M(structRes, samplesInPopList)
Fst <- estimation[1]
M <- estimation[2]
print(paste("Le Fst vaut :", Fst))
print(paste("Le flux de gènes estimé est :", M))